# Create and Invoke Agent via Boto3 SDK

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

## Introduction

In this notebook we show you how to use the `bedrock-agent` and the `bedrock-agent-runtime` boto3 clients to:
- create an agent
- create an action group using an API Schema (vs using function definitions)
- associate the agent with the action group and prepare the agent
- create an agent alias
- invoke the agent

We will use Bedrock's Claude Sonnet using the Boto3 API. 

**Note:** *This notebook can be used in SageMaker Studio or run locally if you setup your AWS credentials.*

#### Prerequisites
This notebook requires permissions to: 
- create and delete Amazon IAM roles
- create, update and invoke AWS Lambda functions 
- create, update and delete Amazon S3 buckets 
- access Amazon Bedrock 

If you are running this notebook without an Admin role, make sure that your role include the following managed policies:
- IAMFullAccess
- AWSLambda_FullAccess
- AmazonS3FullAccess
- AmazonBedrockFullAccess


#### Context
We will demonstrate how to create and invoke an agent for Bedrock using the Boto3 SDK

#### Use case
For this notebook, our agent acts as an assistant for an insurance claims use case. The agent helps the insurance employee to check open claims, identify the details for a specific claim, get open documents for a claim and send reminders to a claim policyholder.

The Agent created can handle the follow tasks or combinations of these in a multi-step process:
- Get Open Claims
- Get Claim Details
- Get Claim Outstanding Documents
- Send Claim reminder

## Notebook setup
Before starting, let's import the required packages and configure the support variables

In [2]:
import logging
import boto3
import time
import zipfile
from io import BytesIO
import json
import uuid
import pprint

In [3]:
# setting logger
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [4]:
# get boto3 clients for required AWS services
sts_client = boto3.client('sts')
iam_client = boto3.client('iam')
s3_client = boto3.client('s3')
lambda_client = boto3.client('lambda')
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

In [5]:
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
region, account_id

('us-west-2', '963354651517')

In [59]:
# Generate random prefix for unique IAM roles, agent name and S3 Bucket and 
# assign variables
suffix = f"{region}-{account_id}"
agent_name = "financial-advisor-agent"
agent_alias_name = "workshop-alias"
bucket_name = f'{agent_name}-{suffix}'
bucket_key = f'{agent_name}-schema.json'
schema_name = 'insurance_claims_agent_openapi_schema.json'
schema_arn = f'arn:aws:s3:::{bucket_name}/{bucket_key}'
bedrock_agent_bedrock_allow_policy_name = f"{agent_name}-allow-{suffix}"
bedrock_agent_s3_allow_policy_name = f"{agent_name}-s3-allow-{suffix}"
bedrock_agent_kb_allow_policy_name = f"{agent_name}-KB-allow-{suffix}"
lambda_role_name = f'{agent_name}-lambda-role-{suffix}'
agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{suffix}'
lambda_code_path_insurance = "lambda_function_insurance.py"
lambda_name_insurance = f'{agent_name}-{suffix}-insurance'
lambda_code_path_financials = "lambda_function_financials.py"
lambda_name_financials = f'{agent_name}-{suffix}-financials'
lambda_dynamodb_policy_name = f'{agent_name}-{suffix}-dynamodb-lambda-policy'
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

# update paramters
knowledge_base_arn = 'arn:aws:bedrock:us-west-2:963354651517:knowledge-base/LLT8CLEFQU'
knowledge_base_id = "LLT8CLEFQU"

### Create S3 bucket and upload API Schema

Agents require an API Schema stored on s3. Let's create an S3 bucket to store the file and upload the file to the newly created bucket

In [8]:
# Create S3 bucket for Open API schema
if region == "us-east-1":
    s3bucket = s3_client.create_bucket(
        Bucket=bucket_name
    )
else:
    s3bucket = s3_client.create_bucket(
        Bucket=bucket_name,
        CreateBucketConfiguration={ 'LocationConstraint': region } 
    )

In [9]:
# Upload Open API schema to this s3 bucket
s3_client.upload_file(schema_name, bucket_name, bucket_key)

### Create Lambda functions for Insurance and Financials Action Group
Let's now create the lambda function required by the agent action group. We first need to create the lambda IAM role and it's policy. After that, we package the lambda function into a ZIP format to create the function

In [ ]:
lambda_dynamodb_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "dynamodb:BatchGetItem",
                "dynamodb:GetItem",
                "dynamodb:Query",
                "dynamodb:Scan",
                "dynamodb:BatchWriteItem",
                "dynamodb:PutItem",
                "dynamodb:UpdateItem"
            ],
            "Resource": "*"
        },
        {
            "Effect": "Allow",
            "Action": [
                "logs:CreateLogStream",
                "logs:PutLogEvents"
            ],
            "Resource": "*"
        },
        {
            "Effect": "Allow",
            "Action": "logs:CreateLogGroup",
            "Resource": "*"
        }
    ]
}
lambda_dynamodb_policy_statement_json = json.dumps(lambda_dynamodb_policy_statement)

agent_kb_schema_policy = iam_client.create_policy(
    PolicyName=lambda_dynamodb_policy_name,
    Description=f"Policy to allow agent to retrieve documents from knowledge base.",
    PolicyDocument=lambda_dynamodb_policy_statement_json
)

In [10]:
# Create IAM Role for the Lambda function
try:
    assume_role_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": "bedrock:InvokeModel",
                "Principal": {
                    "Service": "lambda.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }

    assume_role_policy_document_json = json.dumps(assume_role_policy_document)

    lambda_iam_role = iam_client.create_role(
        RoleName=lambda_role_name,
        AssumeRolePolicyDocument=assume_role_policy_document_json
    )

    # Pause to make sure role is created
    time.sleep(10)
except:
    lambda_iam_role = iam_client.get_role(RoleName=lambda_role_name)

iam_client.attach_role_policy(
    RoleName=lambda_role_name,
    PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
)

iam_client.attach_role_policy(
    RoleName=lambda_role_name,
    PolicyArn=lambda_dynamodb_policy_name['Policy']['Arn']
)


{'ResponseMetadata': {'RequestId': 'bdc74fe1-e89c-44c0-bab2-a2fabbbca260',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sat, 24 Aug 2024 14:05:25 GMT',
   'x-amzn-requestid': 'bdc74fe1-e89c-44c0-bab2-a2fabbbca260',
   'content-type': 'text/xml',
   'content-length': '212'},
  'RetryAttempts': 0}}

Take a look at the Lambda function code that will be used as an Action group for the agent

In [11]:
!pygmentize lambda_function_insurance.py

#!/usr/bin/env python3
# Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
# SPDX-License-Identifier: MIT-0
import json


def get_named_parameter(event, name):
    return next(item for item in event['parameters'] if item['name'] == name)['value']


def get_named_property(event, name):
    return next(
        item for item in
        event['requestBody']['content']['application/json']['properties']
        if item['name'] == name)['value']


def claim_detail(payload):
    claim_id = payload['parameters'][0]['value']
    if claim_id == 'claim-857':
        return {
            "response": {
                "claimId": claim_id,
                "createdDate": "21-Jul-2023",
                "lastActivityDate": "25-Jul-2023",
                "status": "Open",
                "policyType": "Vehicle"
            }
        }
    elif claim_id == 'claim-006':
        return {
            "response": {
                "claimId": claim_id,
                "createdDate": "20-May-2

In [12]:
# Package up the lambda function code
s = BytesIO()
z = zipfile.ZipFile(s, 'w')
z.write(lambda_code_path_insurance)
z.close()
zip_content = s.getvalue()

# Create Lambda Function
lambda_function_insurance = lambda_client.create_function(
    FunctionName=lambda_name_insurance,
    Runtime='python3.12',
    Timeout=180,
    Role=lambda_iam_role['Role']['Arn'],
    Code={'ZipFile': zip_content},
    Handler='lambda_function_insurance.lambda_handler'
)

In [14]:
!pygmentize lambda_function_financials.py

import json
import boto3
import decimal

# DynamoDB boto3 resource and variable
dynamodb = boto3.resource('dynamodb',region_name='us-west-2')
loans_table_name = 'financial-advisor-userdetails'
table = dynamodb.Table(loans_table_name)

def get_named_parameter(event, name):
    return next(item for item in event['parameters'] if item['name'] == name)['value']

def get_named_property(event, name):
    return next(
        item for item in
        event['requestBody']['content']['application/json']['properties']
        if item['name'] == name)['value']
        
def getUserFinances(user_id):
    response = table.get_item(
                Key={'userid': user_id})
    
    return response['Item']      

def checkLoanEligibility(user_id):
    response = table.get_item(
                Key={'userid': user_id})
                
    creditscore = response['Item']['creditscore']
   
    print(creditscore)    
    
    if (creditscore > 600):
        return {
            "response": {
            

In [15]:
# Package up the lambda function code
s = BytesIO()
z = zipfile.ZipFile(s, 'w')
z.write(lambda_code_path_financials)
z.close()
zip_content = s.getvalue()

# Create Lambda Function
lambda_function_financials = lambda_client.create_function(
    FunctionName=lambda_name_financials,
    Runtime='python3.12',
    Timeout=180,
    Role=lambda_iam_role['Role']['Arn'],
    Code={'ZipFile': zip_content},
    Handler='lambda_function_financials.lambda_handler'
)

### Create Agent
We will now create our agent. To do so, we first need to create the agent policies that allow bedrock model invocation  and s3 bucket access. 

In [17]:
# Create IAM policies for agent

bedrock_agent_bedrock_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                f"arn:aws:bedrock:{region}::foundation-model/{model_id}"
            ]
        }
    ]
}

bedrock_policy_json = json.dumps(bedrock_agent_bedrock_allow_policy_statement)

agent_bedrock_policy = iam_client.create_policy(
    PolicyName=bedrock_agent_bedrock_allow_policy_name,
    PolicyDocument=bedrock_policy_json
)



Next, we will create a policy document that allows fetching of the Agent's OpenAPI schema from S3:

In [18]:
bedrock_agent_s3_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AllowAgentAccessOpenAPISchema",
            "Effect": "Allow",
            "Action": ["s3:GetObject"],
            "Resource": [
                schema_arn
            ]
        }
    ]
}


bedrock_agent_s3_json = json.dumps(bedrock_agent_s3_allow_policy_statement)
agent_s3_schema_policy = iam_client.create_policy(
    PolicyName=bedrock_agent_s3_allow_policy_name,
    Description=f"Policy to allow invoke Lambda that was provisioned for it.",
    PolicyDocument=bedrock_agent_s3_json
)

In [19]:
bedrock_agent_kb_retrival_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "bedrock:Retrieve"
            ],
            "Resource": [
                knowledge_base_arn
            ]
        }
    ]
}
bedrock_agent_kb_json = json.dumps(bedrock_agent_kb_retrival_policy_statement)

agent_kb_schema_policy = iam_client.create_policy(
    PolicyName=bedrock_agent_kb_allow_policy_name,
    Description=f"Policy to allow agent to retrieve documents from knowledge base.",
    PolicyDocument=bedrock_agent_kb_json
)

Finally, create a role with the above three policies attached

In [20]:
# Create IAM Role for the agent and attach IAM policies
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [{
          "Effect": "Allow",
          "Principal": {
            "Service": "bedrock.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
    }]
}

assume_role_policy_document_json = json.dumps(assume_role_policy_document)

agent_role = iam_client.create_role(
    RoleName=agent_role_name,
    AssumeRolePolicyDocument=assume_role_policy_document_json
)

# Pause to make sure role is created
time.sleep(10)
    
iam_client.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_bedrock_policy['Policy']['Arn']
)

iam_client.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_s3_schema_policy['Policy']['Arn']
)

iam_client.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_kb_schema_policy['Policy']['Arn']
)

{'ResponseMetadata': {'RequestId': 'f173cfa5-b9d4-432b-a0ed-2930bf33efff',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sat, 24 Aug 2024 14:08:04 GMT',
   'x-amzn-requestid': 'f173cfa5-b9d4-432b-a0ed-2930bf33efff',
   'content-type': 'text/xml',
   'content-length': '212'},
  'RetryAttempts': 0}}

#### Creating Agent
Once the needed IAM role is created, we can use the bedrock agent client to create a new agent. To do so we use the `create_agent` function. It requires an agent name, underlying foundation model and instructions. You can also provide an agent description. Note that the agent created is not yet prepared. We will focus on preparing the agent and then using it to invoke actions and use other APIs

In [21]:
# Create Agent
agent_instruction = """
You are an agent that can handle various queries and tasks related to financial planning including including looking up 
user details, check eligibility of the loan, apply credit card, check claims, insurance details, and share financial product information. If an user asks
about your functionality, provide guidance in natural language 
"""

response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=agent_role['Role']['Arn'],
    description="Agent for handling customer's financial queries",
    idleSessionTTLInSeconds=1800,
    foundationModel=model_id,
    instruction=agent_instruction,
)

Looking at the created agent, we can see its status and agent id

In [22]:
response

{'ResponseMetadata': {'RequestId': '8a280ebe-a9da-4f00-9f43-e6ba92fce4c8',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Sat, 24 Aug 2024 14:08:31 GMT',
   'content-type': 'application/json',
   'content-length': '887',
   'connection': 'keep-alive',
   'x-amzn-requestid': '8a280ebe-a9da-4f00-9f43-e6ba92fce4c8',
   'x-amz-apigw-id': 'dBFm_G1dPHcErwg=',
   'x-amzn-trace-id': 'Root=1-66c9e95f-5200df7a7f618f445bbbee6b'},
  'RetryAttempts': 0},
 'agent': {'agentArn': 'arn:aws:bedrock:us-west-2:963354651517:agent/P0N7MLXVYY',
  'agentId': 'P0N7MLXVYY',
  'agentName': 'financial-advisor-agent',
  'agentResourceRoleArn': 'arn:aws:iam::963354651517:role/AmazonBedrockExecutionRoleForAgents_us-west-2-963354651517',
  'agentStatus': 'CREATING',
  'createdAt': datetime.datetime(2024, 8, 24, 14, 8, 31, 720443, tzinfo=tzlocal()),
  'description': "Agent for handling customer's financial queries",
  'foundationModel': 'anthropic.claude-3-sonnet-20240229-v1:0',
  'idleSessionTTLInSeconds': 1800,

Let's now store the agent id in a local variable to use it on the next steps

In [23]:
agent_id = response['agent']['agentId']
agent_id

'P0N7MLXVYY'

### Create Agent Action Group for Insurance 
We will now create and agent action group that uses the lambda function and API schema files created before.
The `create_agent_action_group` function provides this functionality. We will use `DRAFT` as the agent version since we haven't yet create an agent version or alias. To inform the agent about the action group functionalities, we will provide an action group description containing the functionalities of the action group.

In [24]:
# Pause to make sure agent is created
time.sleep(30)
# Now, we can configure and create an action group here:
agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'lambda': lambda_function_insurance['FunctionArn']
    },
    actionGroupName='ClaimManagementActionGroup',
    apiSchema={
        's3': {
            's3BucketName': bucket_name,
            's3ObjectKey': bucket_key
        }
    },
    description='Actions for listing claims, identifying missing paperwork, sending reminders'
)

In [25]:
agent_action_group_response

{'ResponseMetadata': {'RequestId': '319010ff-c66e-4378-a2bc-05ab442d87d8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sat, 24 Aug 2024 14:09:10 GMT',
   'content-type': 'application/json',
   'content-length': '632',
   'connection': 'keep-alive',
   'x-amzn-requestid': '319010ff-c66e-4378-a2bc-05ab442d87d8',
   'x-amz-apigw-id': 'dBFs4Gb7vHcEs2g=',
   'x-amzn-trace-id': 'Root=1-66c9e985-151f980c11dc6cca2fbd3dc5'},
  'RetryAttempts': 0},
 'agentActionGroup': {'actionGroupExecutor': {'lambda': 'arn:aws:lambda:us-west-2:963354651517:function:financial-advisor-agent-us-west-2-963354651517-insurance'},
  'actionGroupId': 'IECAU4E2FU',
  'actionGroupName': 'ClaimManagementActionGroup',
  'actionGroupState': 'ENABLED',
  'agentId': 'P0N7MLXVYY',
  'agentVersion': 'DRAFT',
  'apiSchema': {'s3': {'s3BucketName': 'financial-advisor-agent-us-west-2-963354651517',
    's3ObjectKey': 'financial-advisor-agent-schema.json'}},
  'createdAt': datetime.datetime(2024, 8, 24, 14, 9, 10, 102450, 

### Allowing Agent to invoke Action Group Lambda
Before using our action group, we need to allow our agent to invoke the lambda function associated to the action group. This is done via resource-based policy. Let's add the resource-based policy to the lambda function created

In [27]:
# Create allow invoke permission on lambda
response = lambda_client.add_permission(
    FunctionName=lambda_name_insurance,
    StatementId='allow_bedrock',
    Action='lambda:InvokeFunction',
    Principal='bedrock.amazonaws.com',
    SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}",
)

## Create Agent Action Group for Financials
We will now create an agent action group that uses the lambda function created earlier. The [`create_agent_action_group`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent/client/create_agent_action_group.html) function provides this functionality. We will use `DRAFT` as the agent version since we haven't yet created an agent version or alias. To inform the agent about the action group capabilities, we provide an action group description.

In this example, we provide the Action Group functionality using a `functionSchema`. You can alternatively provide an `APISchema`. The notebook [02-create-agent-with-api-schema.ipynb](02-create-agent-with-api-schema/02-create-agent-with-api-schema.ipynb) provides an example of that approach.

To define the functions using a function schema, you need to provide the `name`, `description` and `parameters` for each function.

In [34]:
agent_functions = [
    {
       "name": "loan_eligibility",
       "description": "To check the eligibility of the loan for the current user. This function to be called only when the userid is available and agent or model cannot call this function to get the user id. User Id needs to be asked from user only. ",
       "parameters": {
           "userid" : {
              "description": "unqiue login id or user id of the user",
              "required": True,
              "type": "string"
         }
       }
    },
    {
      "name": "get_user_finances",
      "description": "To get the financial details of current user like monthly salary, credit score, assets, investment, etc. This function to be called only when the userid is available and agent or model cannot call this function to get the user id. User Id needs to be asked from user only. ",
      "parameters": {
           "userid" : {
              "description": "unqiue login id or user id of the user",
              "required": True,
              "type": "string"
         }
       }
    },
]

In [35]:
# Now, we can configure and create an action group here:
agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'lambda': lambda_function_financials['FunctionArn']
    },
    actionGroupName='FinancialManagementActionGroup',
    functionSchema={
        'functions': agent_functions
    },
    description="To gather the current user details and loan eligibility criteria"
)

### Allowing Agent to invoke Action Group Lambda
Before using our action group, we need to allow our agent to invoke the lambda function associated to the action group. This is done via resource-based policy. Let's add the resource-based policy to the lambda function created

In [36]:
# Create allow invoke permission on lambda
response = lambda_client.add_permission(
    FunctionName=lambda_name_financials,
    StatementId='allow_bedrock',
    Action='lambda:InvokeFunction',
    Principal='bedrock.amazonaws.com',
    SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}",
)

### Associate agent to the Knowledge Base
Next, we need to associate the agent created with the Knowledge Base for the Bedrock documentation

In [41]:
agent_kb_description = bedrock_agent_client.associate_agent_knowledge_base(
    agentId=agent_id,
    agentVersion='DRAFT',
    description='Use the information in this knowledge base to answer the user queries related to the financial products, investments, queries, etc.',
    knowledgeBaseId=knowledge_base_id 
)

### Preparing Agent
Let's create a DRAFT version of the agent that can be used for internal testing.

In [42]:
agent_prepare = bedrock_agent_client.prepare_agent(agentId=agent_id)
agent_prepare

{'ResponseMetadata': {'RequestId': 'b3218f3a-6345-478c-bc51-6e7df2bf2a70',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Sat, 24 Aug 2024 14:28:35 GMT',
   'content-type': 'application/json',
   'content-length': '119',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'b3218f3a-6345-478c-bc51-6e7df2bf2a70',
   'x-amz-apigw-id': 'dBIjDHRcPHcET-A=',
   'x-amzn-trace-id': 'Root=1-66c9ee13-3bb80ae67ec428ba1fbbe427'},
  'RetryAttempts': 0},
 'agentId': 'P0N7MLXVYY',
 'agentStatus': 'PREPARING',
 'agentVersion': 'DRAFT',
 'preparedAt': datetime.datetime(2024, 8, 24, 14, 28, 35, 322433, tzinfo=tzlocal())}

### Create Agent alias
We will now create an alias of the agent that can be used to deploy the agent.

In [43]:
# Pause to make sure agent is prepared
time.sleep(30)
agent_alias = bedrock_agent_client.create_agent_alias(
    agentId=agent_id,
    agentAliasName=agent_alias_name
)

In [44]:
agent_alias

{'ResponseMetadata': {'RequestId': '0ccfbcc8-abff-4f76-8afc-995dcf3a46bb',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Sat, 24 Aug 2024 14:29:07 GMT',
   'content-type': 'application/json',
   'content-length': '340',
   'connection': 'keep-alive',
   'x-amzn-requestid': '0ccfbcc8-abff-4f76-8afc-995dcf3a46bb',
   'x-amz-apigw-id': 'dBIoBEFSvHcEtJQ=',
   'x-amzn-trace-id': 'Root=1-66c9ee33-2f86e17077607eb45dbd2074'},
  'RetryAttempts': 0},
 'agentAlias': {'agentAliasArn': 'arn:aws:bedrock:us-west-2:963354651517:agent-alias/P0N7MLXVYY/V3UYTX4P1W',
  'agentAliasId': 'V3UYTX4P1W',
  'agentAliasName': 'workshop-alias',
  'agentAliasStatus': 'CREATING',
  'agentId': 'P0N7MLXVYY',
  'createdAt': datetime.datetime(2024, 8, 24, 14, 29, 7, 153331, tzinfo=tzlocal()),
  'routingConfiguration': [{}],
  'updatedAt': datetime.datetime(2024, 8, 24, 14, 29, 7, 153331, tzinfo=tzlocal())}}

### Invoke Agent
Now that we've created the agent, let's use the `bedrock-agent-runtime` client to invoke this agent and perform some tasks.

In [45]:
time.sleep(30)
# Extract the agentAliasId from the response
agent_alias_id = agent_alias['agentAlias']['agentAliasId']

In [46]:
## create a random id for session initiator id
session_id:str = str(uuid.uuid1())
enable_trace:bool = False
end_session:bool = False
# Pause to make sure agent alias is ready
# time.sleep(30)

# invoke the agent API
agentResponse = bedrock_agent_runtime_client.invoke_agent(
    inputText="what are the open claims?",
    agentId=agent_id,
    agentAliasId=agent_alias_id, 
    sessionId=session_id,
    enableTrace=enable_trace, 
    endSession= end_session
)

logger.info(pprint.pprint(agentResponse))

[2024-08-24 14:29:37,382] p1117 {1019905135.py:18} INFO - None


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/json',
                                      'date': 'Sat, 24 Aug 2024 14:29:37 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': '4a603b6a-6225-11ef-b77b-72aa1cd3978e',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': '5b486509-2c72-4c24-b13c-e0fd673c0293'},
                      'HTTPStatusCode': 200,
                      'RequestId': '5b486509-2c72-4c24-b13c-e0fd673c0293',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x7f3d2439a4d0>,
 'contentType': 'application/json',
 'sessionId': '4a603b6a-6225-11ef-b77b-72aa1cd3978e'}


In [47]:
%%time
event_stream = agentResponse['completion']
try:
    for event in event_stream:        
        if 'chunk' in event:
            data = event['chunk']['bytes']
            logger.info(f"Final answer ->\n{data.decode('utf8')}")
            agent_answer = data.decode('utf8')
            end_event_received = True
            # End event indicates that the request finished successfully
        elif 'trace' in event:
            logger.info(json.dumps(event['trace'], indent=2))
        else:
            raise Exception("unexpected event.", event)
except Exception as e:
    raise Exception("unexpected event.", e)

[2024-08-24 14:29:50,699] p1117 {<timed exec>:6} INFO - Final answer ->
Based on the information retrieved, here are the details of the open insurance claims:

Claim ID: claim-006
Policy Holder ID: A945684  
Claim Status: Open
Policy Type: Vehicle
Created Date: 20-May-2023
Last Activity Date: 23-Jul-2023

Claim ID: claim-857 
Policy Holder ID: A645987
Claim Status: Open  
Policy Type: Vehicle
Created Date: 21-Jul-2023
Last Activity Date: 25-Jul-2023

Claim ID: claim-334
Policy Holder ID: A987654
Claim Status: Open
Policy Type: Vehicle  
Created Date: 18-Apr-2023
Last Activity Date: 20-Apr-2023


CPU times: user 0 ns, sys: 4.07 ms, total: 4.07 ms
Wall time: 13.3 s


In [48]:
# And here is the response if you just want to see agent's reply
print(agent_answer)

Based on the information retrieved, here are the details of the open insurance claims:

Claim ID: claim-006
Policy Holder ID: A945684  
Claim Status: Open
Policy Type: Vehicle
Created Date: 20-May-2023
Last Activity Date: 23-Jul-2023

Claim ID: claim-857 
Policy Holder ID: A645987
Claim Status: Open  
Policy Type: Vehicle
Created Date: 21-Jul-2023
Last Activity Date: 25-Jul-2023

Claim ID: claim-334
Policy Holder ID: A987654
Claim Status: Open
Policy Type: Vehicle  
Created Date: 18-Apr-2023
Last Activity Date: 20-Apr-2023


In [49]:
def simple_agent_invoke(input_text, agent_id, agent_alias_id, session_id=None, enable_trace=False, end_session=False):
    agentResponse = bedrock_agent_runtime_client.invoke_agent(
        inputText=input_text,
        agentId=agent_id,
        agentAliasId=agent_alias_id, 
        sessionId=session_id,
        enableTrace=enable_trace, 
        endSession= end_session
    )
    
    event_stream = agentResponse['completion']
    try:
        for event in event_stream:        
            if 'chunk' in event:
                data = event['chunk']['bytes']
                logger.info(f"Final answer ->\n{data.decode('utf8')}")
                agent_answer = data.decode('utf8')
                end_event_received = True
                # End event indicates that the request finished successfully
            elif 'trace' in event:
                logger.info(json.dumps(event['trace'], indent=2))
            else:
                raise Exception("unexpected event.", event)
    except Exception as e:
        raise Exception("unexpected event.", e)

In [52]:
# query = "tell me about claim-857"
query = "I want to buy a car. Am I eligible for the same?"
simple_agent_invoke(query, agent_id, agent_alias_id, session_id)

[2024-08-24 14:39:03,738] p1117 {3357547408.py:16} INFO - Final answer ->
To check your eligibility for a car loan, I will need your user ID or login ID. Could you please provide me with your user ID?


In [57]:
query = "rohan is my userid"
simple_agent_invoke(query, agent_id, agent_alias_id, session_id)

[2024-08-24 16:29:54,217] p1117 {3357547408.py:16} INFO - Final answer ->
Based on Rohan's financial details:
- Credit score of 750 (good)
- Monthly income of Rs. 100,000
- Assets worth Rs. 100,000

The following loan options may be suitable for him:



- Personal loans: Rohan can avail of unsecured personal loans from banks and NBFCs based on his income and credit score. The loan amount and interest rate will depend on his eligibility.

- Credit cards: With a good credit score, Rohan can apply for credit cards from various banks and NBFCs. The credit limit will depend on his income and existing debt obligations.

- Vehicle loans: If Rohan plans to purchase a new or used vehicle, he can apply for a vehicle loan. The loan amount will depend on the vehicle's price, and he may need to provide a down payment.






Additionally, with assets worth Rs. 100,000, Rohan may be able to avail of secured loans like:

- Loan against property: If Rohan owns a property, he can pledge it as collateral

In [58]:
query = "how my investment portfolio looks like?"
simple_agent_invoke(query, agent_id, agent_alias_id, session_id)

[2024-08-24 16:30:47,142] p1117 {3357547408.py:16} INFO - Final answer ->
Based on the <REDACTED> function results, your investment portfolio is as follows:

Investment Portfolio:
- Debt: 10%
- Equity: 60% 
- Mutual Funds: 30%

The function results did not provide the specific amounts invested in each category, but showed that you have total investments worth Rs. 30,000.


In [51]:
simple_agent_invoke("send reminders for all open claims that have missing paperwork", agent_id, agent_alias_id, session_id)

[2024-08-24 14:30:21,745] p1117 {3357547408.py:16} INFO - Final answer ->
I have sent reminders for all open insurance claims that had pending documents:

For claim-006, a reminder was sent to upload AccidentImages with tracking ID 50e8400-e29b-41d4-a716-446655440000.

For claim-857, a reminder was sent to upload DriverLicense and VehicleRegistration with tracking ID 50e8400-e29b-41d4-a716-446655440000.

No reminder was sent for claim-334 as there were no pending documents for that claim.


### Clean up (optional)
The next steps are optional and demonstrate how to delete our agent. To delete the agent we need to:
1. update the action group to disable it
2. delete agent action group
3. delete agent alias
4. delete agent
5. delete lambda function
6. empty created s3 bucket
7. delete s3 bucket

In [ ]:
 # This is not needed, you can delete agent successfully after deleting alias only
# Additionaly, you need to disable it first

action_group_id = agent_action_group_response['agentActionGroup']['actionGroupId']
action_group_name = agent_action_group_response['agentActionGroup']['actionGroupName']

response = bedrock_agent_client.update_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupId= action_group_id,
    actionGroupName=action_group_name,
    actionGroupExecutor={
        'lambda': lambda_function['FunctionArn']
    },
    apiSchema={
        's3': {
            's3BucketName': bucket_name,
            's3ObjectKey': bucket_key
        }
    },
    actionGroupState='DISABLED',
)

action_group_deletion = bedrock_agent_client.delete_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupId= action_group_id
)

In [ ]:
agent_alias_deletion = bedrock_agent_client.delete_agent_alias(
    agentId=agent_id,
    agentAliasId=agent_alias['agentAlias']['agentAliasId']
)

In [ ]:
agent_deletion = bedrock_agent_client.delete_agent(
    agentId=agent_id
)

In [ ]:
# Delete Lambda function
lambda_client.delete_function(
    FunctionName=lambda_name
)

In [ ]:
# Empty and delete S3 Bucket

objects = s3_client.list_objects(Bucket=bucket_name)  
if 'Contents' in objects:
    for obj in objects['Contents']:
        s3_client.delete_object(Bucket=bucket_name, Key=obj['Key']) 
s3_client.delete_bucket(Bucket=bucket_name)

In [ ]:
# Delete IAM Roles and policies

for policy in [bedrock_agent_bedrock_allow_policy_name, bedrock_agent_s3_allow_policy_name]:
    iam_client.detach_role_policy(RoleName=agent_role_name, PolicyArn=f'arn:aws:iam::{account_id}:policy/{policy}')
    
iam_client.detach_role_policy(RoleName=lambda_role_name, PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole')

for role_name in [agent_role_name, lambda_role_name]:
    iam_client.delete_role(
        RoleName=role_name
    )

for policy in [agent_bedrock_policy, agent_s3_schema_policy]:
    iam_client.delete_policy(
        PolicyArn=policy['Policy']['Arn']
)

## Conclusion
We have now experimented with using `boto3` SDK to create, invoke and delete an agent.

### Take aways
- Adapt this notebook to create new agents for your application

## Thank You